<a href="https://colab.research.google.com/github/Pardhu156/Recommender-system/blob/main/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boltzmann Machines

## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [ ]:
# movies=pd.read_csv('ml-1m/movies.dat',sep='::',header=None,engine='python',encoding='latin-1')
# users=pd.read_csv('ml-1m/users.dat',sep='::',header=None,engine='python',encoding='latin-1')
# ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')

## Preparing the training set and the test set

In [ ]:
training_set=pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set=np.array(training_set,dtype='int')
test_set=pd.read_csv('ml-100k/u1.test',delimiter='\t')
test_set=np.array(test_set,dtype='int')

## Getting the number of users and movies

In [ ]:
nb_users=int(max(max(training_set[ : ,0]),max(test_set[ : ,0])))
nb_movies=int(max(max(training_set[ : ,1]),max(test_set[ : ,1])))

## Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
  new_data=[]
  for id_users in range(1,nb_users+1):
    id_movies=data[ : ,1][data[ : ,0]==id_users]
    id_ratings=data[ : ,2][data[ : ,0]==id_users]
    ratings=np.zeros(nb_movies)
    ratings[id_movies-1]=id_ratings
    new_data.append(list(ratings))
  return new_data

In [ ]:
training_set=convert(training_set)
test_set=convert(test_set)

## Converting the data into Torch tensors

In [ ]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [ ]:
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1
test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

## Creating the architecture of the Neural Network

In [ ]:
class RBM():
  def __init__(self,nv,nh):
    self.w=torch.randn(nh,nv)
    self.a=torch.randn(1,nh)
    self.b=torch.randn(1,nv)
  def sample_h(self,x):
    wx=torch.mm(x,self.w.t())
    function=wx+self.a.expand_as(wx)
    p_h_given_v=torch.sigmoid(function)
    return p_h_given_v,torch.bernoulli(p_h_given_v)
  def sample_v(self,y):
    wy=torch.mm(y,self.w)
    function=wy+self.b.expand_as(wy)
    p_v_given_h=torch.sigmoid(function)
    return p_v_given_h,torch.bernoulli(p_v_given_h)
  def train(self,v0,vk,ph0,phk):
    self.w=self.w+torch.mm(v0.t(),ph0).t()-torch.mm(vk.t(),phk).t()
    self.a=self.a+torch.sum((ph0-phk),0)
    self.b=self.b+torch.sum((v0-vk),0)
nv=nb_movies #also use len(training_set[0])
nh=100
rbm=RBM(nv,nh)
batch_size=100

## Training the RBM

In [ ]:
for epochs in range(1,11): #epochs=10
    train_loss=0
    s=0.
    for id_user in range(0,nb_users-batch_size,batch_size):
      v0=training_set[id_user:id_user+batch_size]
      vk=training_set[id_user:id_user+batch_size]
      ph0,_=rbm.sample_h(v0)
      for k in range(11):
        _,hk=rbm.sample_h(vk)
        _,vk=rbm.sample_v(hk)
        vk[v0<0]=v0[v0<0]
      phk,_=rbm.sample_h(vk)
      rbm.train(v0,vk,ph0,phk)
      train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
      s=s+1.
    print('epoch:'+str(epochs)+' loss:'+str(train_loss/s))

epoch:1 loss:tensor(0.3408)
epoch:2 loss:tensor(0.2534)
epoch:3 loss:tensor(0.2380)
epoch:4 loss:tensor(0.2279)
epoch:5 loss:tensor(0.2459)
epoch:6 loss:tensor(0.2479)
epoch:7 loss:tensor(0.2443)
epoch:8 loss:tensor(0.2459)
epoch:9 loss:tensor(0.2449)
epoch:10 loss:tensor(0.2442)


## Testing the RBM

In [ ]:
test_loss=0
s=0.
for id_user in range(nb_users):
  v=training_set[id_user:id_user+1]
  vt=test_set[id_user:id_user+1]
  if len(vt[vt>=0])>0:
    _,h=rbm.sample_h(v)
    _,v=rbm.sample_v(h)
    test_loss=test_loss+torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
    s=s+1.
print('loss: '+str(test_loss/s))

loss: tensor(0.2497)
